In [ ]:
# 필요한 라이브러리들을 설치합니다.
!pip install torchsummary scikit-learn joblib tqdm -q

# 데이터셋을 GitHub에서 복제합니다.
!git clone https://github.com/hbcbh1999/recaptcha-dataset

# 데이터셋에서 이번 챌린지에 사용하지 않는 폴더를 제거합니다.
!rm -rf ./recaptcha-dataset/Large/Mountain/
!rm -rf ./recaptcha-dataset/Large/Other/
!rm -rf ./recaptcha-dataset/Large/readme.txt
print("데이터셋 준비 완료.")

In [ ]:
# --- 라이브러리 임포트 ---
import os
import glob
import copy
import time
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms
from torch.utils.data import DataLoader, Dataset, Subset
from PIL import Image
import numpy as np
import pandas as pd
import joblib
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
import warnings
from tqdm.notebook import tqdm

# 불필요한 경고 메시지는 무시합니다.
warnings.filterwarnings('ignore')

# PyTorch 및 GPU 사용 가능 여부를 확인합니다.
print(f"PyTorch Version: {torch.__version__}")
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

In [ ]:
class ImageDataset(Dataset):
    """이미지 경로와 라벨을 받아 PyTorch가 사용할 수 있는 데이터셋 형태로 변환합니다."""
    def __init__(self, image_paths, labels, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert('RGB')
        label = self.labels[idx]
        if self.transform:
            image = self.transform(image)
        return image, label

def train_model(model, dataloaders, criterion, optimizer, num_epochs=15, patience=5):
    """
    모델 학습을 위한 메인 루프입니다. Early Stopping 기능이 포함되어 있습니다.

    Args:
        patience (int): 검증 세트의 성능이 이 epoch만큼 개선되지 않으면 학습을 조기 종료합니다.
    """
    since = time.time()
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    epochs_no_improve = 0

    model = model.to(device)
    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # 학습 모드
            else:
                model.eval()   # 평가 모드

            running_loss, running_corrects = 0.0, 0
            for inputs, labels in dataloaders[phase]:
                inputs, labels = inputs.to(device), labels.to(device)
                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)
                    _, preds = torch.max(outputs, 1)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)
            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            # 가장 성능이 좋았던 모델의 가중치를 저장 (Early Stopping)
            if phase == 'val':
                if epoch_acc > best_acc:
                    best_acc = epoch_acc
                    best_model_wts = copy.deepcopy(model.state_dict())
                    epochs_no_improve = 0
                else:
                    epochs_no_improve += 1

        # 정해진 patience 동안 성능 향상이 없으면 학습 종료
        if epochs_no_improve >= patience:
            print(f'\nEarly stopping! Validation accuracy has not improved for {patience} epochs.')
            break
        print()

    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Best val Acc: {best_acc:4f}')
    model.load_state_dict(best_model_wts)
    return model

class FineTunedFeatureExtractor:
    """
    파인튜닝된 모델을 불러와 마지막 분류 레이어를 제거하고,
    중간 단계의 깊은 피처(Deep Feature)를 추출하는 역할을 합니다.
    """
    def __init__(self, model_path):
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        print(f"FineTunedFeatureExtractor: Using device: {self.device}")

        # 저장된 모델을 불러옵니다.
        full_model = torch.load(model_path, map_location=self.device)

        # 마지막 fully-connected layer를 제거하여 피처 추출기로 사용합니다.
        self.feature_extractor = nn.Sequential(*list(full_model.children())[:-1])
        self.feature_extractor.to(self.device)
        self.feature_extractor.eval() # 평가 모드로 설정

        # 모델 학습 시 사용했던 것과 동일한 이미지 변환(전처리)을 정의합니다.
        self.transform = transforms.Compose([
            transforms.Resize(256),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])

    def extract(self, image_path):
        """단일 이미지 경로를 입력받아 딥피처 벡터를 추출합니다."""
        try:
            img = Image.open(image_path).convert('RGB')
            img_tensor = self.transform(img).unsqueeze(0).to(self.device)
            with torch.no_grad(): # 그래디언트 계산 비활성화
                feature = self.feature_extractor(img_tensor)
            return feature.squeeze().cpu().numpy()
        except Exception as e:
            print(f"Warning: {image_path} 피처 추출 실패 - {e}")
            return None

In [ ]:
def prepare_dataloaders_and_split_data(dataset_path='./recaptcha-dataset/Large/'):
    """데이터셋을 로드하고 학습/테스트용으로 분리한 뒤, 데이터로더를 생성합니다."""
    print("--- [1/4] 데이터 준비 시작 ---")
    target_labels = ['Bicycle', 'Bridge', 'Bus', 'Car', 'Chimney', 'Crosswalk', 'Hydrant', 'Motorcycle', 'Palm', 'Traffic Light']
    label_to_idx = {label: i for i, label in enumerate(target_labels)}

    all_image_paths = sorted([p for label in target_labels for p in glob.glob(os.path.join(dataset_path, label, '*.png'))])
    if not all_image_paths:
        print(f"CRITICAL ERROR: '{dataset_path}' 경로에서 이미지를 찾을 수 없습니다.")
        return None, None, None, None, None

    all_str_labels = [os.path.basename(os.path.dirname(p)) for p in all_image_paths]
    all_int_labels = [label_to_idx[name] for name in all_str_labels]

    # Stratify 옵션으로 클래스 비율을 유지하며 80/20 분할
    train_paths, test_paths, train_labels, test_labels = train_test_split(
        all_image_paths, all_int_labels, test_size=0.2, random_state=42, stratify=all_int_labels
    )
    print(f"총 데이터: {len(all_image_paths)}, 학습 데이터: {len(train_paths)}, 테스트 데이터: {len(test_paths)}")

    # 학습 시 데이터 증강(Augmentation)을 위한 변환 정의
    data_transforms = {
        'train': transforms.Compose([
            transforms.RandomResizedCrop(224),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ]),
        'val': transforms.Compose([
            transforms.Resize(256),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ]),
    }

    # 학습 데이터셋(80%) 내부에서 다시 학습용과 검증용으로 분리
    full_train_dataset = ImageDataset(train_paths, train_labels, transform=None)
    train_indices, val_indices, _, _ = train_test_split(
        range(len(full_train_dataset)), full_train_dataset.labels, test_size=0.2, random_state=42, stratify=full_train_dataset.labels
    )

    train_subset = Subset(full_train_dataset, train_indices)
    val_subset = Subset(full_train_dataset, val_indices)

    # 데이터셋에 맞는 변환(augmentation) 적용
    train_subset.dataset = copy.deepcopy(full_train_dataset); train_subset.dataset.transform = data_transforms['train']
    val_subset.dataset.transform = data_transforms['val']

    dataloaders = {
        'train': DataLoader(train_subset, batch_size=32, shuffle=True, num_workers=2),
        'val': DataLoader(val_subset, batch_size=32, shuffle=False, num_workers=2)
    }

    # 다음 단계를 위해 필요한 결과물들을 반환
    return dataloaders, (train_paths, train_labels), (test_paths, test_labels), target_labels

# 1단계 실행
dataloaders, train_data, test_data, target_labels = prepare_dataloaders_and_split_data()

In [ ]:
def fine_tune_and_save_model(dataloaders, target_labels):
    """ResNet 모델을 파인튜닝하고, 가장 성능이 좋았던 모델을 저장합니다."""
    print("\n--- [2/4] 모델 파인튜닝 시작 ---")
    if dataloaders is None:
        print("데이터로더가 준비되지 않아 파인튜닝을 건너뜁니다.")
        return None

    # ImageNet으로 사전학습된 ResNet18 모델 로드
    model_ft = models.resnet18(weights='IMAGENET1K_V1')
    # 마지막 레이어의 출력 뉴런 수를 우리 클래스 개수에 맞게 수정
    num_ftrs = model_ft.fc.in_features
    model_ft.fc = nn.Linear(num_ftrs, len(target_labels))

    optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)
    criterion = nn.CrossEntropyLoss()

    # 모델 학습 실행
    finetuned_model = train_model(model_ft, dataloaders, criterion, optimizer_ft, num_epochs=25, patience=5)

    # 최종 모델 저장
    finetuned_model_path = 'c2_finetuned_model.pt'
    torch.save(finetuned_model, finetuned_model_path)
    print(f"파인튜닝된 모델 저장 완료: '{finetuned_model_path}'")
    return finetuned_model_path

# 2단계 실행
if dataloaders:
    finetuned_model_path = fine_tune_and_save_model(dataloaders, target_labels)

In [ ]:
def build_and_optimize_feature_db(model_path, train_data):
    """파인튜닝된 모델로 피처를 추출하고, PCA 최적화를 통해 최종 DB를 구축합니다."""
    print("\n--- [3/4] DB 구축 및 최적화 시작 ---")
    if model_path is None:
        print("모델 경로가 없어 DB 구축을 건너뜁니다.")
        return

    train_paths, train_labels = train_data
    feature_extractor = FineTunedFeatureExtractor(model_path=model_path)

    # 학습 데이터(80%) 전체에서 딥피처 추출
    db_features, db_labels = [], []
    for path, label in tqdm(zip(train_paths, train_labels), total=len(train_paths), desc="DB 피처 추출 중"):
        feature = feature_extractor.extract(path)
        if feature is not None:
            db_features.append(feature)
            db_labels.append(label)

    if not db_features:
        print("CRITICAL ERROR: DB 피처 구축에 실패했습니다.")
        return

    X_raw, y_db = np.array(db_features), np.array(db_labels)

    # 피처 스케일링 (평균 0, 분산 1로 조정)
    scaler = StandardScaler().fit(X_raw)
    X_scaled = scaler.transform(X_raw)

    # 교차 검증을 통해 최적의 PCA 차원 수 탐색
    param_grid = {'n_components': [64, 128, 256]}
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    best_score, best_n_comp = -1, 0

    print("\n최적 PCA 차원 수 탐색 중...")
    for n_comp in param_grid['n_components']:
        accuracies = []
        for train_idx, val_idx in cv.split(X_scaled, y_db):
            X_train_fold, X_val_fold = X_scaled[train_idx], X_scaled[val_idx]
            y_train_fold, y_val_fold = y_db[train_idx], y_db[val_idx]

            pca = PCA(n_components=n_comp).fit(X_train_fold)
            knn = KNeighborsClassifier(n_neighbors=1).fit(pca.transform(X_train_fold), y_train_fold)
            accuracies.append(accuracy_score(y_val_fold, knn.predict(pca.transform(X_val_fold))))

        avg_accuracy = np.mean(accuracies)
        print(f"  - PCA(n_components={n_comp}) | CV Accuracy: {avg_accuracy:.4f}")
        if avg_accuracy > best_score:
            best_score, best_n_comp = avg_accuracy, n_comp

    print(f"\n최적 PCA 차원 수 결정: {best_n_comp}")

    # 최적 차원 수로 최종 PCA 모델 학습 및 DB 변환
    final_pca = PCA(n_components=best_n_comp).fit(X_scaled)
    final_db_index = final_pca.transform(X_scaled)

    # 최종 결과물들 저장
    joblib.dump(scaler, 'c2_scaler.pkl')
    joblib.dump(final_pca, 'c2_pca_model.pkl')
    np.save('c2_db_index.npy', final_db_index)
    np.save('c2_db_labels.npy', y_db)
    print("DB 구축 및 최적화 모델 저장이 완료되었습니다.")


# 3단계 실행
if 'finetuned_model_path' in locals() and finetuned_model_path:
    build_and_optimize_feature_db(finetuned_model_path, train_data)

In [ ]:
def evaluate_on_test_data(test_data, target_labels, models_dir='./'):
    """저장된 모든 모델을 사용하여 테스트 데이터셋의 성능을 평가합니다."""
    print("\n--- [4/4] 최종 평가 시작 ---")
    try:
        model_path = os.path.join(models_dir, 'c2_finetuned_model.pt')
        scaler = joblib.load(os.path.join(models_dir, 'c2_scaler.pkl'))
        pca = joblib.load(os.path.join(models_dir, 'c2_pca_model.pkl'))
        db_index = np.load(os.path.join(models_dir, 'c2_db_index.npy'))
        db_labels = np.load(os.path.join(models_dir, 'c2_db_labels.npy'))
    except FileNotFoundError as e:
        print(f"오류: 평가에 필요한 모델 파일 로드 실패 - {e}. 이전 단계를 먼저 실행해주세요.")
        return

    test_paths, true_int_labels = test_data
    feature_extractor = FineTunedFeatureExtractor(model_path=model_path)
    knn_matcher = KNeighborsClassifier(n_neighbors=10, metric='cosine').fit(db_index, db_labels)

    task1_results, task2_results, predicted_labels = [], [], []
    for img_path in tqdm(test_paths, desc="테스트 데이터 평가 중"):
        query_raw = feature_extractor.extract(img_path)
        if query_raw is None: continue

        # DB 구축 시와 동일한 전처리 적용
        query_scaled = scaler.transform(query_raw.reshape(1, -1))
        query_final = pca.transform(query_scaled)

        # K-NN을 이용해 DB에서 가장 유사한 10개 이미지 검색
        _, indices = knn_matcher.kneighbors(query_final)
        top_k_labels_int = db_labels[indices.flatten()]

        # Task 1: 가장 유사한 이미지의 클래스를 예측값으로 사용
        predicted_class_int = top_k_labels_int[0]
        predicted_labels.append(predicted_class_int)

        # 결과 저장용 리스트
        task1_results.append([os.path.basename(img_path), target_labels[predicted_class_int]])
        task2_results.append([os.path.basename(img_path)] + [target_labels[l] for l in top_k_labels_int])

    # 성능 리포트 출력
    print("\n--- 최종 성능 평가 결과 ---")
    print(classification_report(true_int_labels, predicted_labels, target_names=target_labels))

    # 제출용 CSV 파일 저장
    pd.DataFrame(task1_results).to_csv('c2_task1_results.csv', header=False, index=False)
    pd.DataFrame(task2_results).to_csv('c2_task2_results.csv', header=False, index=False)
    print("\n평가 완료. 'c2_task1_results.csv', 'c2_task2_results.csv' 파일이 생성되었습니다.")


# 4단계 실행
if 'test_data' in locals() and test_data[0]:
    evaluate_on_test_data(test_data, target_labels)